In [1]:

pip install google-api-python-client nltk


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [2]:
import re

# Extract the video ID using regex
def extract_video_id(url):
    match = re.search(r'(?:v=|youtu.be/)([a-zA-Z0-9_-]+)', url)
    if match:
        return match.group(1)
    else:
        raise ValueError("Invalid YouTube URL")

# Example Usage:
video_url = 'https://youtu.be/mWQ_422IWLY?si=ew_rIv7QkVNt8GyK'
video_id = extract_video_id(video_url)
print(video_id)


mWQ_422IWLY


In [ ]:
import os
import googleapiclient.discovery
import googleapiclient.errors
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

# Set up NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Set up API key
api_key = 'YOUR_API_KEY'  # Replace with your API key

# Extract video ID using regex
def extract_video_id(url):
    match = re.search(r'(?:v=|youtu.be/)([a-zA-Z0-9_-]+)', url)
    if match:
        return match.group(1)
    else:
        raise ValueError("Invalid YouTube URL")

# Function to get comments from a YouTube video
def get_video_comments(video_id, api_key):
    youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)
    comments = []
    next_page_token = None
    
    while True:
        try:
            # Get the comments from the video
            request = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                textFormat="plainText",
                pageToken=next_page_token
            )
            response = request.execute()
    
            # Extract the comments
            for item in response['items']:
                comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
                comments.append(comment)
    
            # Check if there is another page of comments
            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break  # No more comments to fetch
    
        except googleapiclient.errors.HttpError as e:
            print(f"An error occurred: {e}")
            break
    
    return comments

# Function to process the comments (Stemming and Lemmatization)
def process_comments(comments):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    
    processed_comments = []
    
    for comment in comments:
        # Tokenize the comment
        words = word_tokenize(comment.lower())
        
        # Remove stopwords
        words = [word for word in words if word not in stop_words and word.isalpha()]
        
        # Apply Stemming and Lemmatization
        stemmed_words = [stemmer.stem(word) for word in words]
        lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]
        
        # Join the words back together
        processed_comment = ' '.join(lemmatized_words)
        processed_comments.append(processed_comment)
    
    return processed_comments

# Main Execution
if __name__ == "__main__":
    video_url = 'https://youtu.be/mWQ_422IWLY?si=ew_rIv7QkVNt8GyK'  # Your provided YouTube video URL
    video_id = extract_video_id(video_url)  # Extract video ID

    # Extract comments
    comments = get_video_comments(video_id, api_key)
    
    # Process comments
    processed_comments = process_comments(comments)
    
    # Output processed comments
    for i, comment in enumerate(processed_comments, 1):
        print(f"Comment {i}: {comment}")
